# **Price prediction on California's coast** #

Elaborate a model that enables prediciton of median housing prices per block in california.

---

## **Frame the problem** ##
Outputs will be fet to another ML algorithm evaluating if investing in a California's area is worth. It is really important to get accurate prices.
A full dataset of california median prices per block is given. It includes median price per block + many other metrics.

---

### **Buisness objective** ###
Build a ML algorithm to determine areas that are worth investing in. This restricts to California.

---

### **Use case** ###
Outputs will be fet to another ML algorithm evaluating if investing in a California's area is worth.

---

### **Current solutions?** ###
Manual evaluation by a team of experts. If price data does not exists, it is estimated using complex rules. Current precision us +- 30%

---

### **Problem framing** ###
- Clear supervised problem
- Batch learning
- Multiple, univariate Regression problem

---

### **How to measure performance?** ###
Norm : L2 or L1. Depends on outliers
$$RMSE(X,h) = \sqrt{\frac{1}{m_i}\sum_{i=1}^m(h(X_i)-y_i)^2}$$
$$MAE(X,h) = \frac{1}{m_i}\sum_{i=1}^m|h(X_i)-y_i|$$

---

### **Is performance measure aligned with buisness objective?** ###
Yes as it will allow to know whether we fall under the 30% average error mark.

---

### **Minimum performance for production** ###
Fall under the 30% error mark 

---

### **Comparable or existing solutions** ###

None, for exercice purpose

---

### **Human expertise available?** ###

The experts already working to evaluate housing prices.

---

### **How to solve problem manually** ###

Ask the experts

---

### **List assumptions** ###

Currently, we assume the downstream algorithm will use the prices as a continuous value. If for instance it was a category (*low price, medium price, high price*), the precision is less important. Only the category.
==> The problem becomes a **classification task**

---

### **Evaluate assumptions** ###

For exercice purpose, we assume the task is indeed a regression.

---


## **Get the data** ##

### **What data and how much?** ###
We need the california district housing prices from the SatLib dataset

---

### Where to get the data? ###

---

### How much space will the data take ###

---

### Legal obligations ###

---

### get authorizations ###

---

### Create a workspace ###

---

### Get the data ###

---

### Convert the data without changing it ###

---

### Anonimize / protect the data ###

---

### Check size & type ###

---

### Sample a test set and never look at it ###

## Explore the data ##

### Create a copy ###

### Create a notebook to recode exploration ###

### Study each attribute ###

#### Name ####
#### Type ####
#### Missing values (%) ####
#### Noisiness ####
#### Usefulness ####
#### Distribution ####

### If supervised : what attribute? ###

### Visualise the data ###

### Explore correlations ###

### How to solve manually ###

### Study promising transformations to apply before ###

### Extra data required? ###

### Document what has been learned ###


## Prepare the data ##

### Clean the data ###

#### Remove outliers? ####
#### Fill missing values or drop columns / rows ####

### Select useful features (drop some?) ###

### Feature engineering? ###
#### Discretize continuous ####
#### Decompose ####
#### Add promising transformations ####
#### Aggregate features into new ones ####

### Scale features ###
#### Standardize or normalize ####


## Shortlist models ##

### Try and train many quick and dirty models (linear, Bayes, SVM, neural nets, random forests, etc...) ###

### Measure performance (use N-folds cross validation) ###

### Analize significant variables for each model ###

### Analyze errors models make and what data would a human need to avoid these? ###

### Quick round of feature selection / engineering ###

### Quickly iterate the previous steps (one or two times) ###

### Select a few models (3 to 5) preferring models that make different types of errors ###

## Fine tune ##

### Fine tune hyperparameters using cross-validation ###
- Treat data transforms as hyperparameters especially if not sure about them
- Unless very few hyperparameters, prefer random search over grid search. If training is long, use optimizations

### Try running models together (often gives better results) ###

### Once confindent, validate the model against the test set ###

### Measure performance (use N-folds cross validation) ###

### Analize significant variables for each model ###

### Analyze errors models make and what data would a human need to avoid these? ###

### Quick round of feature selection / engineering ###

### Quickly iterate the previous steps (one or two times) ###

### Select a few models (3 to 5) preferring models that make different types of errors ###